In [ ]:
%%spark

In [2]:
song_df = spark.read.json("s3a://udacity-dend/song_data/*/*/*/*.json")

VBox()

In [ ]:
song_df.count()

In [3]:
song_df.printSchema()

VBox()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [4]:
filcols = ["song_id", "title", "artist_id", "year", "duration"]
song_table = song_df[filcols]

VBox()

In [5]:
#song_table.write.partitionBy("year","artist_id").parquet("s3://sparkify-songs-data/songstable/")

VBox()

In [6]:
song_data = song_df[filcols].dropDuplicates()

VBox()

In [ ]:
song_data.createOrReplaceTempView("Songs")

In [ ]:
spark.sql("select count(*) from songs").show()

In [7]:
filartistcols = ["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]
artist_data = song_df[filartistcols].dropDuplicates()
#artist_data.write.parquet("s3://sparkify-songs-data/artisttable/")

VBox()

In [ ]:
artist_data.createOrReplaceTempView("Artists")

In [ ]:
spark.sql("select count(*) from Artists").show()

In [8]:
log_df = spark.read.json("s3a://udacity-dend/log_data/2018/11/*.json")
#log_data = df.write.parquet("s3://sparkify-log-data/log_data/")

VBox()

In [9]:
log_df.printSchema()

VBox()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [ ]:
log_df.count()

In [10]:
#df = df.filter("page == 'NextSong'").show()
log_df = log_df[log_df['page']=='NextSong']

VBox()

In [11]:
filusercols = ["userId","firstName","lastName","gender","level"]
#user_data = df[filusercols].dropDuplicates()
#user_data.count()
#user_data.createOrReplaceTempView("Users")
#spark.sql("select count(*) from Users").show()

VBox()

In [12]:
user_data = log_df[filusercols].dropDuplicates()

VBox()

In [ ]:
user_data.count()

In [ ]:
user_data.createOrReplaceTempView("Users")

In [ ]:
#user_data.write.parquet("s3://sparkify-songs-data/usertable/")

In [13]:
from pyspark.sql.functions import udf,to_timestamp,hour,dayofmonth,weekofyear,month,year,dayofweek,date_format

VBox()

In [14]:
import datetime

VBox()

In [15]:
get_timestamp = udf(lambda x: datetime.datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S'))

VBox()

In [16]:
log_df = log_df.withColumn('start_time',to_timestamp(get_timestamp(log_df.ts)))\
       .withColumn('hour',hour(to_timestamp(get_timestamp(log_df.ts))))\
       .withColumn('day',dayofmonth(to_timestamp(get_timestamp(log_df.ts))))\
       .withColumn('week',weekofyear(to_timestamp(get_timestamp(log_df.ts))))\
       .withColumn('month',month(to_timestamp(get_timestamp(log_df.ts))))\
       .withColumn('year',year(to_timestamp(get_timestamp(log_df.ts))))\
       .withColumn('weekday',dayofweek(to_timestamp(get_timestamp(log_df.ts))))

VBox()

In [17]:
time_table = log_df[['start_time','hour','day','week','month','year','weekday']].dropDuplicates()

VBox()

In [ ]:
time_table.count()

In [18]:
#time_table.write.partitionBy("year","month").parquet(("s3://sparkify-songs-data/timetable/"))

VBox()

In [ ]:
spark.sql('select * from songs LIMIT 20')

In [19]:
songplay_df = log_df.join(song_df,(log_df.song == song_df.title)&(log_df.artist == song_df.artist_name)&(log_df.length == song_df.duration)&(log_df.page == 'NextSong'),"inner")

VBox()

In [ ]:
songplay_df.count()

In [ ]:
songplay_df.show()

In [20]:
filsongplaycols = songplay_df[['ts','userId','level','song_id','artist_id','sessionId','location','userAgent']].dropDuplicates()

VBox()

In [21]:
filsongplaycols.show()

VBox()

+-------------+------+-----+------------------+------------------+---------+--------------------+--------------------+
|           ts|userId|level|           song_id|         artist_id|sessionId|            location|           userAgent|
+-------------+------+-----+------------------+------------------+---------+--------------------+--------------------+
|1543514154796|    82| paid|SOBONKR12A58A7A7E0|AR5E44Z1187B9A1D74|     1017|Atlanta-Sandy Spr...|"Mozilla/5.0 (Mac...|
|1542237247796|    49| paid|SOWGZFG12A8151AF41|ARC8CQZ1187B98DECA|      576|San Francisco-Oak...|Mozilla/5.0 (Wind...|
|1543009329796|    85| paid|SONEGWF12A8C133022|ARI75KH1187FB4C77C|      891|       Red Bluff, CA|"Mozilla/5.0 (Mac...|
|1542043597796|    73| paid|SOCJWPI12A8C13D357|ARRB3GQ1187FB52D14|      294|Tampa-St. Petersb...|"Mozilla/5.0 (Mac...|
|1542496421796|    70| paid|SOKUCXB12A6D4FAC0C|ARHO39G1187FB4E31B|      113|San Antonio-New B...|Mozilla/5.0 (Wind...|
|1543596208796|    33| free|SOKSREW12A6D4F926D|A

In [23]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.functions import row_number

VBox()

In [24]:
songplay_df = songplay_df.withColumn('test',lit(0))\
                        .withColumn('start_time',to_timestamp(get_timestamp(songplay_df.ts)))\
                        .withColumn('user_id',songplay_df.userId)\
                        .withColumn('level',songplay_df.level)\
                        .withColumn('song_id',songplay_df.song_id)\
                        .withColumn('artist_id',songplay_df.artist_id)\
                        .withColumn('session_id',songplay_df.sessionId)\
                        .withColumn('location',songplay_df.location)\
                        .withColumn('month',month(to_timestamp(get_timestamp(songplay_df.ts))))\
                        .withColumn('year',year(to_timestamp(get_timestamp(songplay_df.ts))))

VBox()

In [28]:
songplay_df = songplay_df.withColumn('songplay_id',row_number().over(Window.partitionBy(songplay_df.test).orderBy(songplay_df.user_id)))\
                        .withColumn('start_time',to_timestamp(get_timestamp(songplay_df.ts)))\
                        .withColumn('user_id',songplay_df.userId)\
                        .withColumn('level',songplay_df.level)\
                        .withColumn('song_id',songplay_df.song_id)\
                        .withColumn('artist_id',songplay_df.artist_id)\
                        .withColumn('session_id',songplay_df.sessionId)\
                        .withColumn('location',songplay_df.location)\
                        .withColumn('month',month(to_timestamp(get_timestamp(songplay_df.ts))))\
                        .withColumn('year',year(to_timestamp(get_timestamp(songplay_df.ts))))

VBox()

In [29]:
filsongplaycols = songplay_df[['test','songplay_id','start_time','user_id','level','song_id','artist_id','session_id','location']].dropDuplicates()

VBox()

In [ ]:
filsongplaycols.count()

In [31]:
filsongplaycols.select(['songplay_id','start_time','user_id','level','song_id','artist_id','session_id','location']).sort('songplay_id').show()

VBox()

+-----------+-------------------+-------+-----+------------------+------------------+----------+--------------------+
|songplay_id|         start_time|user_id|level|           song_id|         artist_id|session_id|            location|
+-----------+-------------------+-------+-----+------------------+------------------+----------+--------------------+
|          1|2018-11-02 17:31:45|     10| free|SOHTKMO12AB01843B0|AR5EYTL1187B98EDA0|       182|Washington-Arling...|
|          2|2018-11-20 15:50:01|    100| free|SOHTKMO12AB01843B0|AR5EYTL1187B98EDA0|       595|New York-Newark-J...|
|          3|2018-11-07 07:58:28|    100| free|SODTPBM12A8C1339D7|AR4OH581187B9B7157|       301|New York-Newark-J...|
|          4|2018-11-14 20:36:44|    101| free|SOBONKR12A58A7A7E0|AR5E44Z1187B9A1D74|       603|New Orleans-Metai...|
|          5|2018-11-22 13:21:20|    101| free|SOUHTWB12A8C13BA4D|AR96LYR1187B9ABABD|       790|New Orleans-Metai...|
|          6|2018-11-12 21:22:01|     12| free|SOARUPP12